## Get the prediction dataset

In [123]:
import pandas as pd
import numpy as np

In [164]:
ebitda = pd.read_csv("../clean_data/ebitda_complete.csv")
cogs = pd.read_csv("../clean_data/cogs_complete.csv")
revenue = pd.read_csv("../clean_data/revenue_complete.csv")
market = pd.read_csv("../input_data/market_data.csv")
final_df = pd.read_csv("../clean_data/final_df.csv", index_col=0)

In [125]:
prediction_df = final_df[final_df["Year"]>=2022]

In [126]:
prediction_df.head()

,Company,Device,Market,SubMarket,Year,Value,Spendings,MarketSpends,SubMarketSpends,DevSpends,Cogs,Cogs_1,Cogs_2,Revenue,Revenue_1,Revenue_2,Ebitda,Ebitda_1,Ebitda_2
4,ABB,Amplifier/Comparator,Industrial,Power & Energy,2022,10.040442,257.635,15244.412,2525.705,5469.003,19712.000000,19407.000000,18123.0,29446.00,28945.00,26134.0,4477.00,4641.00,2668.0
5,ABB,Amplifier/Comparator,Industrial,Power & Energy,2023,10.927740,291.701,16317.564,2875.164,5133.007,9879.678571,19712.000000,19407.0,32174.52,29446.00,28945.0,5807.68,4477.00,4641.0
6,ABB,Amplifier/Comparator,Industrial,Power & Energy,2024,11.659800,322.803,17578.984,3481.204,5225.000,10122.197857,9879.678571,19712.0,32970.35,32174.52,29446.0,5943.65,5807.68,4477.0
11,ABB,Analog Application Specific IC,Industrial,Power & Energy,2022,29.153250,646.449,15244.412,2525.705,58394.007,19712.000000,19407.000000,18123.0,29446.00,28945.00,26134.0,4477.00,4641.00,2668.0
12,ABB,Analog Application Specific IC,Industrial,Power & Energy,2023,33.304200,736.031,16317.564,2875.164,51983.007,9879.678571,19712.000000,19407.0,32174.52,29446.00,28945.0,5807.68,4477.00,4641.0


In [127]:
prediction_df = prediction_df.rename(columns={"Value": "CompanySpend"})

In [128]:
prediction_df.replace({2022:2025, 2023:2026, 2024:2027}, inplace=True)

### Financials

In [129]:
ebitda = ebitda[ebitda["Year"]>=2023]
cogs = cogs[cogs["Year"]>=2023]
revenue = revenue[revenue["Year"]>=2023]

In [130]:
comp_map = pd.read_csv("../input_data/financials_map.csv", delimiter= ",", encoding='unicode_escape', index_col = 0)
comp_map

,Company Name from Spend Tracker,Company Tag from S&P Global
0,ABB,ABB Ltd (SWX:ABBN)
1,Abbott Laboratories,Abbott Laboratories (NYSE:ABT)
2,Acer,Acer Incorporated (TWSE:2353)
3,Adient,Adient plc (NYSE:ADNT)
4,Advantest,Advantest Corporation (TSE:6857)
...,...,...
263,Yaskawa Electric Corporation,YASKAWA Electric Corporation (TSE:6506)
264,Yokogawa Electric,Yokogawa Electric Corporation (TSE:6841)
265,Zebra Technologies,Zebra Technologies Corporation (NasdaqGS:ZBRA)
266,ZF Group,ZF Group Inc.


In [131]:
replacement_dict = dict(zip(comp_map["Company Tag from S&P Global"], comp_map["Company Name from Spend Tracker"]))
cogs.Company = cogs.Company.replace(replacement_dict)
ebitda.Company = ebitda.Company.replace(replacement_dict)
revenue.Company = revenue.Company.replace(replacement_dict)

In [132]:
prediction_df.drop("Cogs", axis = 1, inplace=True)
prediction_df=prediction_df.merge(cogs, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Value":"Cogs"}, axis=1)

In [133]:
cogs["Year"] = cogs["Year"]+1

In [134]:
prediction_df.drop("Cogs_1", axis = 1, inplace=True)
prediction_df=prediction_df.merge(cogs, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Value":"Cogs_1"}, axis=1)

In [135]:
cogs["Year"] = cogs["Year"]+1

In [136]:
prediction_df.drop("Cogs_2", axis = 1, inplace=True)
prediction_df=prediction_df.merge(cogs, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Value":"Cogs_2"}, axis=1)

In [137]:
prediction_df.drop("Ebitda", axis = 1, inplace=True)
prediction_df=prediction_df.merge(ebitda, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Ebitda":"ebitda"}, axis=1)

In [138]:
ebitda["Year"] = ebitda["Year"]+1

In [139]:
prediction_df.drop("Ebitda_1", axis = 1, inplace=True)
prediction_df=prediction_df.merge(ebitda, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Ebitda":"Ebitda_1"}, axis=1)

In [140]:
ebitda["Year"] = ebitda["Year"]+1

In [141]:
prediction_df.drop("Ebitda_2", axis = 1, inplace=True)
prediction_df=prediction_df.merge(ebitda, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Ebitda":"Ebitda_2"}, axis=1)

In [142]:
prediction_df.drop("Revenue", axis = 1, inplace=True)
prediction_df=prediction_df.merge(revenue, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Revenue":"revenue"}, axis=1)

In [143]:
revenue["Year"] = revenue["Year"]+1

In [144]:
prediction_df.drop("Revenue_1", axis = 1, inplace=True)
prediction_df=prediction_df.merge(revenue, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Revenue":"Revenue_1"}, axis=1)

In [145]:
revenue["Year"] = revenue["Year"]+1

In [146]:
prediction_df.drop("Revenue_2", axis = 1, inplace=True)
prediction_df=prediction_df.merge(revenue, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Revenue":"Revenue_2"}, axis=1)

In [147]:
prediction_df = prediction_df.rename({"ebitda":"Ebitda", "revenue":"Revenue"}, axis=1)

In [148]:
prediction_df[(prediction_df.Company == "Facebook (Meta)") & (prediction_df.Market == "Consumer")]

,Company,Device,Market,SubMarket,Year,CompanySpend,Spendings,MarketSpends,SubMarketSpends,DevSpends,Cogs,Cogs_1,Cogs_2,Ebitda,Ebitda_1,Ebitda_2,Revenue,Revenue_1,Revenue_2
2556,Facebook (Meta),Amplifier/Comparator,Consumer,Other Consumer,2025,2.7093,116.000,7755.155,2131.102,5469.003,432.249286,423.256071,414.262857,94700.02,83816.96,70266.82,167677.79,70266.82,133151.09
2557,Facebook (Meta),Amplifier/Comparator,Consumer,Other Consumer,2026,2.0539,88.000,6292.004,1757.005,5133.007,441.242500,432.249286,423.256071,106657.76,94700.02,83816.96,184547.58,167677.79,70266.82
2558,Facebook (Meta),Amplifier/Comparator,Consumer,Other Consumer,2027,2.2004,93.000,6430.006,1936.006,5225.000,450.235714,441.242500,432.249286,119613.89,106657.76,94700.02,202653.00,184547.58,167677.79
2565,Facebook (Meta),Analog Application Specific IC,Consumer,Other Consumer,2025,6.5614,1194.104,7755.155,2131.102,58394.007,432.249286,423.256071,414.262857,94700.02,83816.96,70266.82,167677.79,70266.82,133151.09
2566,Facebook (Meta),Analog Application Specific IC,Consumer,Other Consumer,2026,5.4970,1039.002,6292.004,1757.005,51983.007,441.242500,432.249286,423.256071,106657.76,94700.02,83816.96,184547.58,167677.79,70266.82
2567,Facebook (Meta),Analog Application Specific IC,Consumer,Other Consumer,2027,5.5946,1222.001,6430.006,1936.006,51955.005,450.235714,441.242500,432.249286,119613.89,106657.76,94700.02,202653.00,184547.58,167677.79
2574,Facebook (Meta),Data Converters,Consumer,Other Consumer,2025,4.6802,281.000,7755.155,2131.102,5342.999,432.249286,423.256071,414.262857,94700.02,83816.96,70266.82,167677.79,70266.82,133151.09
2575,Facebook (Meta),Data Converters,Consumer,Other Consumer,2026,2.8639,170.001,6292.004,1757.005,4949.004,441.242500,432.249286,423.256071,106657.76,94700.02,83816.96,184547.58,167677.79,70266.82
2576,Facebook (Meta),Data Converters,Consumer,Other Consumer,2027,2.5978,152.002,6430.006,1936.006,5028.001,450.235714,441.242500,432.249286,119613.89,106657.76,94700.02,202653.00,184547.58,167677.79
2583,Facebook (Meta),Interface,Consumer,Other Consumer,2025,1.3482,32.000,7755.155,2131.102,4342.007,432.249286,423.256071,414.262857,94700.02,83816.96,70266.82,167677.79,70266.82,133151.09


### Market

In [165]:
market = market[market["Region"]=="Worldwide"]

In [167]:
mapping_dict = {('Automotive Electronics Categories', 'ADAS'): ('Automotive', 'Auto ADAS'),
    ('Automotive Electronics Categories', 'Body & Convenience'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Chassis & Safety'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Connectivity & Telematics'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Hybrid & Electric Drive Train'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Infotainment & Cluster'): ('Automotive', 'Auto Infotainment'),
    ('Automotive Electronics Categories', 'Other Automotive'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Powertrain & Vehicle Dynamics'): ('Automotive', 'Auto Powertrain'),
    ('Computing & Data Storage Categories', 'Data Center Servers'): ('Computer Platforms', 'Data Center Servers'),
    ('Computing & Data Storage Categories', 'Desktop PCs'): ('Computer Platforms', 'Desktop PCs'),
    ('Computing & Data Storage Categories', 'Flash Storage Cards'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Computing & Data Storage Categories', 'Hard Disk Drives'): ('Computer Peripherals & Storage', 'HDD'),
    ('Computing & Data Storage Categories', 'Notebook PCs'): ('Computer Platforms', 'Notebook PCs'),
    ('Computing & Data Storage Categories', 'Other Computing'): ('Computer Platforms', 'Other Computer Products'),
    ('Computing & Data Storage Categories', 'Other Data Storage'): ('Computer Peripherals & Storage', 'Other Storage'),
    ('Computing & Data Storage Categories', 'Other Peripherals'): ('Computer Peripherals & Storage', 'Other Peripherals'),
    ('Computing & Data Storage Categories', 'Smart Cards'): ('Computer Peripherals & Storage', 'Smart Cards'),
    ('Computing & Data Storage Categories', 'Solid-State Drives'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Computing & Data Storage Categories', 'Tablet PCs'): ('Computer Platforms', 'Tablet PCs'),
    ('Computing & Data Storage Categories', 'USB Flash Drive'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Consumer Electronics Categories', 'Fitness & Wellness Wearable Electronics'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'LCD TV'): ('Consumer', 'TV'),
    ('Consumer Electronics Categories', 'Major Home Appliances'): ('Consumer', 'Appliance'),
    ('Consumer Electronics Categories', 'OLED TV'): ('Consumer', 'TV'),
    ('Consumer Electronics Categories', 'Other Audio/Video'): ('Consumer', 'Audio'),
    ('Consumer Electronics Categories', 'Other Consumer Electronics'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'Set-Top Boxes'): ('Consumer', 'STB'),
    ('Consumer Electronics Categories', 'Smart Speakers & Digital Assistants'): ('Consumer', 'Connected Consumer'),
    ('Consumer Electronics Categories', 'Smart Watches'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'VR Headsets'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'Video Game Consoles'): ('Consumer', 'Video Games'),
    ('Industrial Electronics Categories', 'Automation'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Building & Home Control'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Lighting'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Medical Electronics'): ('Industrial', 'Medical'),
    ('Industrial Electronics Categories', 'Military & Civil Aerospace'): ('Industrial', 'Military/Aerospace'),
    ('Industrial Electronics Categories', 'Other Industrial'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Power & Energy'): ('Industrial', 'Power & Energy'),
    ('Industrial Electronics Categories', 'Security & Video Surveillance'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Test & Measurement'): ('Industrial', 'Test & Measurement'),
    ('Wired Communications Categories', 'Broadcast & Streaming Video'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Cable Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Cable CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Carrier Ethernet Switches & Routers'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'DSL Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'DSL CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Data Center Network Switches'): ('Wired Communications', 'DC Network & Threat Mitigation'),
    ('Wired Communications Categories', 'Enterprise Ethernet Switches & Routers'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'Enterprise UC & Voice'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'FTTH Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'FTTH CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Low-Tier Consumer/SOHO Routers'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'Optical Equipment'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Other Wired Communications'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Threat Mitigation Products'): ('Wired Communications', 'DC Network & Threat Mitigation'),
    ('Wireless Communications Categories', 'Gray Market Handsets'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'High-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Low-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'M2M Modules'): ('Wireless Communications', 'Other Wireless'),
    ('Wireless Communications Categories', 'Media Tablets'): ('Wireless Communications', 'Media Tablets'),
    ('Wireless Communications Categories', 'Mid-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Mobile Comm Infrastructure'): ('Wireless Communications', 'Infrastructure'),
    ('Wireless Communications Categories', 'Mobile Phone (ULCH, Entry, Feature)'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Other Wireless Communications'): ('Wireless Communications', 'Other Wireless'),
    ('Wireless Communications Categories', 'Wireless LAN Equipment'): ('Wireless Communications', 'Infrastructure')}

In [168]:
#Nb of keys
print(len(mapping_dict.keys()))

#Rename the market and submarket

def remapping(market, submarket):
    key = (market, submarket)
    return mapping_dict[key][0], mapping_dict[key][1]

market.Market, market.Submarket = zip(*market.apply(lambda x : remapping(x.Market, x.Submarket), axis = 1))
market.rename(columns = {"Submarket" : "SubMarket"}, inplace = True)
market.drop("Region", axis=1, inplace = True)

65


In [169]:
market = market.groupby(["Market", "SubMarket", "Device", "Year"]).sum().reset_index()

In [162]:
market[(market.Market == "Consumer") & (market.SubMarket == "Other Consumer") & (market.Device == "Interface")]

,Market,SubMarket,Device,Year,Market size
1760,Consumer,Other Consumer,Interface,2008,101.281
1761,Consumer,Other Consumer,Interface,2009,85.324
1762,Consumer,Other Consumer,Interface,2010,87.143
1763,Consumer,Other Consumer,Interface,2011,83.143
1764,Consumer,Other Consumer,Interface,2012,47.688
1765,Consumer,Other Consumer,Interface,2013,37.962
1766,Consumer,Other Consumer,Interface,2014,31.664
1767,Consumer,Other Consumer,Interface,2015,25.628
1768,Consumer,Other Consumer,Interface,2016,15.821
1769,Consumer,Other Consumer,Interface,2017,24.251


In [153]:
marketspendings = market[["Market", "Year", "Market size"]].groupby(["Market", "Year"]).sum().reset_index()
submarketspendings = market[["SubMarket", "Year", "Market size"]].groupby(["SubMarket", "Year"]).sum().reset_index()
devicespendings = market[["Device", "Year", "Market size"]].groupby(["Device", "Year"]).sum().reset_index()

In [154]:
prediction_df = prediction_df.drop("Spendings", axis = 1)
prediction_df=prediction_df.merge(market, how="left", on=["Market", "SubMarket", "Device", "Year"])
prediction_df = prediction_df.rename({"Market size":"Spendings"}, axis=1)

In [155]:
prediction_df.drop(["DevSpends", "MarketSpends", "SubMarketSpends"], axis=1, inplace=True)

In [156]:
prediction_df = prediction_df.merge(marketspendings, how = 'left', on = ["Market", "Year"])
prediction_df.rename(columns = {"Market size" : "MarketSpends"}, inplace = True)
prediction_df = prediction_df.merge(submarketspendings, how = 'left', on = ["SubMarket", "Year"])
prediction_df.rename(columns = {"Market size" : "SubMarketSpends"}, inplace = True)
prediction_df = prediction_df.merge(devicespendings, how = 'left', on = ["Device", "Year"])
prediction_df.rename(columns = {"Market size" : "DevSpends"}, inplace = True)
prediction_df

,Company,Device,Market,SubMarket,Year,CompanySpend,Cogs,Cogs_1,Cogs_2,Ebitda,Ebitda_1,Ebitda_2,Revenue,Revenue_1,Revenue_2,Spendings,MarketSpends,SubMarketSpends,DevSpends
0,ABB,Amplifier/Comparator,Industrial,Power & Energy,2025,10.040442,10364.717143,10122.197857,9879.678571,6271.240000,5943.650000,5807.68,34510.780000,32970.350000,32174.52,349.156,18718.436,3921.473,5641.009
1,ABB,Amplifier/Comparator,Industrial,Power & Energy,2026,10.927740,10607.236429,10364.717143,10122.197857,6997.000000,6271.240000,5943.65,36835.000000,34510.780000,32970.35,387.420,20062.488,4447.796,5829.007
2,ABB,Amplifier/Comparator,Industrial,Power & Energy,2027,11.659800,10849.755714,10607.236429,10364.717143,7210.000000,6997.000000,6271.24,38700.250000,36835.000000,34510.78,435.299,21590.113,4933.962,6035.001
3,ABB,Analog Application Specific IC,Industrial,Power & Energy,2025,29.153250,10364.717143,10122.197857,9879.678571,6271.240000,5943.650000,5807.68,34510.780000,32970.350000,32174.52,1032.715,18718.436,3921.473,57413.002
4,ABB,Analog Application Specific IC,Industrial,Power & Energy,2026,33.304200,10607.236429,10364.717143,10122.197857,6997.000000,6271.240000,5943.65,36835.000000,34510.780000,32970.35,1191.535,20062.488,4447.796,58899.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10550,iRobot,Interface,Consumer,Appliance,2026,8.350700,70910.041071,67334.039286,63758.037500,-23414.371429,-92.790000,-104.38,86546.660714,1177.430000,1076.24,59.001,7496.004,627.003,4850.009
10551,iRobot,Interface,Consumer,Appliance,2027,5.781400,74486.042857,70910.041071,67334.039286,-28093.885714,-23414.371429,-92.79,91136.100000,86546.660714,1177.43,58.000,7425.003,626.000,5159.004
10552,iRobot,Voltage Regulator/Reference,Consumer,Appliance,2025,23.717500,67334.039286,63758.037500,60182.035714,-92.790000,-104.380000,-175.67,1177.430000,1076.240000,990.27,432.000,7535.006,643.001,18326.012
10553,iRobot,Voltage Regulator/Reference,Consumer,Appliance,2026,23.067700,70910.041071,67334.039286,63758.037500,-23414.371429,-92.790000,-104.38,86546.660714,1177.430000,1076.24,439.001,7496.004,627.003,19135.005


In [158]:
prediction_df.to_csv("../clean_data/prediction_df.csv")

### New Target

In [202]:
new_pred = pd.read_csv("../clean_data/newtarget_df.csv", index_col=0)
new_pred = new_pred[new_pred["Year"]>=2021]
new_pred.drop('Target', axis = 1, inplace = True)
new_pred.replace({2021:2024, 2022:2025, 2023:2026, 2024:2027}, inplace=True)
new_pred = new_pred.iloc[:, :5]

In [203]:
comp_map = pd.read_csv("../input_data/financials_map.csv", delimiter= ",", encoding='unicode_escape', index_col = 0)
comp_map
comp_map = comp_map.set_index("Company Tag from S&P Global").to_dict()["Company Name from Spend Tracker"]
companies_available = comp_map.keys()

In [204]:
def mapping_companies(x):
    if x in companies_available:
        return comp_map[x]
    else :
        return None

In [205]:
ebitda = pd.read_csv("../clean_data/ebitda_complete.csv")
ebitda.Company = ebitda.Company.apply(mapping_companies)
ebitda = ebitda[~ebitda['Company'].isnull()]
ebitda["Ebitda_1"] = ebitda.groupby('Company')['Ebitda'].shift(1)
ebitda["Ebitda_2"] = ebitda.groupby('Company')['Ebitda'].shift(2)
remove_ebitda = ebitda[(ebitda.Ebitda_2 < 0) | (ebitda.Ebitda < 0) | (ebitda.Ebitda_1 < 0)].Company.unique()

In [206]:
cogs = pd.read_csv("../clean_data/cogs_complete.csv")
cogs.Company = cogs.Company.apply(mapping_companies)
cogs = cogs[~cogs['Company'].isnull()]
cogs.rename(columns = {"Value" : "Cogs"}, inplace = True)
cogs["Cogs_1"] = cogs.groupby('Company')['Cogs'].shift(1)
cogs["Cogs_2"] = cogs.groupby('Company')['Cogs'].shift(2)
remove_cogs = cogs[(cogs.Cogs_2 < 0) | (cogs.Cogs < 0) | (cogs.Cogs_1 < 0)].Company.unique()
remove_cogs

array(['Wabtec', 'Hewlett-Packard', 'Johnson Controls', 'Schlumberger',
       'LG Display', 'Nabtesco Corporation'], dtype=object)

In [207]:
revenue = pd.read_csv("../clean_data/revenue_complete.csv")
revenue.Company = revenue.Company.apply(mapping_companies)
revenue = revenue[~revenue['Company'].isnull()]
revenue["Revenue_1"] = revenue.groupby('Company')['Revenue'].shift(1)
revenue["Revenue_2"] = revenue.groupby('Company')['Revenue'].shift(2)
remove_revenue = revenue[(revenue.Revenue_2 < 0) | (revenue.Revenue < 0) | (revenue.Revenue_1 < 0)].Company.unique()
revenue

,Company,Year,Revenue,Revenue_1,Revenue_2
0,ABB,2016,24929.00,NaN,NaN
1,ABB,2017,25196.00,24929.00,NaN
2,ABB,2018,27662.00,25196.00,24929.00
3,ABB,2019,27978.00,27662.00,25196.00
4,ABB,2020,26134.00,27978.00,27662.00
...,...,...,...,...,...
2539,Transsion,2023,8270.25,6371.40,6756.40
2540,Transsion,2024,9718.26,8270.25,6371.40
2541,Transsion,2025,11490.58,9718.26,8270.25
2542,Transsion,2026,10273.06,11490.58,9718.26


In [208]:
new_pred = new_pred.merge(cogs, how = "left", on = ["Company", "Year"]).merge(revenue, how = "left", on = ["Company", "Year"]).merge(ebitda, how = "left", on = ["Company", "Year"])
new_pred = new_pred[~new_pred['Cogs'].isnull()].reset_index(drop = True)
new_pred = new_pred[~new_pred['Ebitda'].isnull()].reset_index(drop = True)
new_pred = new_pred[~new_pred['Ebitda_1'].isnull()].reset_index(drop = True)
new_pred = new_pred[~new_pred['Ebitda_2'].isnull()].reset_index(drop = True)
new_pred = new_pred[~new_pred.Company.isin(remove_ebitda)]
new_pred = new_pred[~new_pred.Company.isin(remove_cogs)]
new_pred = new_pred[~new_pred.Company.isin(remove_revenue)]
new_pred

,Company,Device,Market,SubMarket,Year,Cogs,Cogs_1,Cogs_2,Revenue,Revenue_1,Revenue_2,Ebitda,Ebitda_1,Ebitda_2
0,ABB,Amplifier/Comparator,Industrial,Power & Energy,2024,10122.197857,9879.678571,19712.000000,32970.35,32174.52,29446.00,5943.65,5807.680000,4477.000000
1,ABB,Amplifier/Comparator,Industrial,Power & Energy,2025,10364.717143,10122.197857,9879.678571,34510.78,32970.35,32174.52,6271.24,5943.650000,5807.680000
2,ABB,Amplifier/Comparator,Industrial,Power & Energy,2026,10607.236429,10364.717143,10122.197857,36835.00,34510.78,32970.35,6997.00,6271.240000,5943.650000
3,ABB,Amplifier/Comparator,Industrial,Power & Energy,2027,10849.755714,10607.236429,10364.717143,38700.25,36835.00,34510.78,7210.00,6997.000000,6271.240000
4,ABB,Analog Application Specific IC,Industrial,Power & Energy,2024,10122.197857,9879.678571,19712.000000,32970.35,32174.52,29446.00,5943.65,5807.680000,4477.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11494,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Infrastructure,2027,10389.264286,9638.288929,8887.313571,6364.00,5980.00,5260.88,1493.00,1360.000000,1171.030000
11495,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2024,8136.338214,7385.362857,3157.000000,4671.88,4536.76,5781.00,916.76,3668.885714,1140.000000
11496,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2025,8887.313571,8136.338214,7385.362857,5260.88,4671.88,4536.76,1171.03,916.760000,3668.885714
11497,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2026,9638.288929,8887.313571,8136.338214,5980.00,5260.88,4671.88,1360.00,1171.030000,916.760000


In [209]:
market_spend = pd.read_csv("../input_data/market_data.csv")
market_spend

,Market,Submarket,Device,Year,Region,Market size
0,Automotive Electronics Categories,ADAS,Amplifier/Comparator,2008,Americas,0.424
1,Automotive Electronics Categories,ADAS,Amplifier/Comparator,2008,Asia & Oceania (excl. Japan),0.246
2,Automotive Electronics Categories,ADAS,Amplifier/Comparator,2008,EMEA,0.678
3,Automotive Electronics Categories,ADAS,Amplifier/Comparator,2008,Japan,0.421
4,Automotive Electronics Categories,ADAS,Amplifier/Comparator,2008,Worldwide,1.769
...,...,...,...,...,...,...
32495,Wireless Communications Categories,Wireless LAN Equipment,Voltage Regulator/Reference,2027,Americas,0.000
32496,Wireless Communications Categories,Wireless LAN Equipment,Voltage Regulator/Reference,2027,Asia & Oceania (excl. Japan),0.000
32497,Wireless Communications Categories,Wireless LAN Equipment,Voltage Regulator/Reference,2027,EMEA,0.000
32498,Wireless Communications Categories,Wireless LAN Equipment,Voltage Regulator/Reference,2027,Japan,0.000


In [210]:
mapping_dict = {('Automotive Electronics Categories', 'ADAS'): ('Automotive', 'Auto ADAS'),
    ('Automotive Electronics Categories', 'Body & Convenience'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Chassis & Safety'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Connectivity & Telematics'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Hybrid & Electric Drive Train'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Infotainment & Cluster'): ('Automotive', 'Auto Infotainment'),
    ('Automotive Electronics Categories', 'Other Automotive'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Powertrain & Vehicle Dynamics'): ('Automotive', 'Auto Powertrain'),
    ('Computing & Data Storage Categories', 'Data Center Servers'): ('Computer Platforms', 'Data Center Servers'),
    ('Computing & Data Storage Categories', 'Desktop PCs'): ('Computer Platforms', 'Desktop PCs'),
    ('Computing & Data Storage Categories', 'Flash Storage Cards'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Computing & Data Storage Categories', 'Hard Disk Drives'): ('Computer Peripherals & Storage', 'HDD'),
    ('Computing & Data Storage Categories', 'Notebook PCs'): ('Computer Platforms', 'Notebook PCs'),
    ('Computing & Data Storage Categories', 'Other Computing'): ('Computer Platforms', 'Other Computer Products'),
    ('Computing & Data Storage Categories', 'Other Data Storage'): ('Computer Peripherals & Storage', 'Other Storage'),
    ('Computing & Data Storage Categories', 'Other Peripherals'): ('Computer Peripherals & Storage', 'Other Peripherals'),
    ('Computing & Data Storage Categories', 'Smart Cards'): ('Computer Peripherals & Storage', 'Smart Cards'),
    ('Computing & Data Storage Categories', 'Solid-State Drives'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Computing & Data Storage Categories', 'Tablet PCs'): ('Computer Platforms', 'Tablet PCs'),
    ('Computing & Data Storage Categories', 'USB Flash Drive'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Consumer Electronics Categories', 'Fitness & Wellness Wearable Electronics'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'LCD TV'): ('Consumer', 'TV'),
    ('Consumer Electronics Categories', 'Major Home Appliances'): ('Consumer', 'Appliance'),
    ('Consumer Electronics Categories', 'OLED TV'): ('Consumer', 'TV'),
    ('Consumer Electronics Categories', 'Other Audio/Video'): ('Consumer', 'Audio'),
    ('Consumer Electronics Categories', 'Other Consumer Electronics'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'Set-Top Boxes'): ('Consumer', 'STB'),
    ('Consumer Electronics Categories', 'Smart Speakers & Digital Assistants'): ('Consumer', 'Connected Consumer'),
    ('Consumer Electronics Categories', 'Smart Watches'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'VR Headsets'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'Video Game Consoles'): ('Consumer', 'Video Games'),
    ('Industrial Electronics Categories', 'Automation'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Building & Home Control'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Lighting'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Medical Electronics'): ('Industrial', 'Medical'),
    ('Industrial Electronics Categories', 'Military & Civil Aerospace'): ('Industrial', 'Military/Aerospace'),
    ('Industrial Electronics Categories', 'Other Industrial'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Power & Energy'): ('Industrial', 'Power & Energy'),
    ('Industrial Electronics Categories', 'Security & Video Surveillance'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Test & Measurement'): ('Industrial', 'Test & Measurement'),
    ('Wired Communications Categories', 'Broadcast & Streaming Video'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Cable Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Cable CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Carrier Ethernet Switches & Routers'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'DSL Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'DSL CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Data Center Network Switches'): ('Wired Communications', 'DC Network & Threat Mitigation'),
    ('Wired Communications Categories', 'Enterprise Ethernet Switches & Routers'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'Enterprise UC & Voice'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'FTTH Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'FTTH CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Low-Tier Consumer/SOHO Routers'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'Optical Equipment'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Other Wired Communications'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Threat Mitigation Products'): ('Wired Communications', 'DC Network & Threat Mitigation'),
    ('Wireless Communications Categories', 'Gray Market Handsets'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'High-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Low-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'M2M Modules'): ('Wireless Communications', 'Other Wireless'),
    ('Wireless Communications Categories', 'Media Tablets'): ('Wireless Communications', 'Media Tablets'),
    ('Wireless Communications Categories', 'Mid-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Mobile Comm Infrastructure'): ('Wireless Communications', 'Infrastructure'),
    ('Wireless Communications Categories', 'Mobile Phone (ULCH, Entry, Feature)'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Other Wireless Communications'): ('Wireless Communications', 'Other Wireless'),
    ('Wireless Communications Categories', 'Wireless LAN Equipment'): ('Wireless Communications', 'Infrastructure')}

In [211]:
dict_map = {('Automotive Electronics Categories', 'ADAS'): ('Automotive', 'Auto ADAS'), ('Automotive Electronics Categories', 'Body & Convenience'): ('Automotive', 'Connected Consumer'), ('Automotive Electronics Categories', 'Chassis & Safety'): ('Automotive', 'Test & Measurement'), ('Automotive Electronics Categories', 'Connectivity & Telematics'): ('Automotive', 'Test & Measurement'), ('Automotive Electronics Categories', 'Hybrid & Electric Drive Train'): ('Automotive', 'DC Network & Threat Mitigation'), ('Automotive Electronics Categories', 'Infotainment & Cluster'): ('Automotive', 'Auto Infotainment'), ('Automotive Electronics Categories', 'Other Automotive'): ('Automotive', 'Other Storage'), ('Automotive Electronics Categories', 'Powertrain & Vehicle Dynamics'): ('Automotive', 'Power & Energy'), ('Computing & Data Storage Categories', 'Data Center Servers'): ('Computer Peripherals & Storage', 'Data Center Servers'), ('Computing & Data Storage Categories', 'Desktop PCs'): ('Computer Peripherals & Storage', 'Desktop PCs'), ('Computing & Data Storage Categories', 'Flash Storage Cards'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'), ('Computing & Data Storage Categories', 'Hard Disk Drives'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'), ('Computing & Data Storage Categories', 'Notebook PCs'): ('Computer Peripherals & Storage', 'Notebook PCs'), ('Computing & Data Storage Categories', 'Other Computing'): ('Computer Peripherals & Storage', 'Other Computer Products'), ('Computing & Data Storage Categories', 'Other Data Storage'): ('Computer Peripherals & Storage', 'Other Storage'), ('Computing & Data Storage Categories', 'Other Peripherals'): ('Computer Peripherals & Storage', 'Other Peripherals'), ('Computing & Data Storage Categories', 'Smart Cards'): ('Computer Peripherals & Storage', 'Smart Cards'), ('Computing & Data Storage Categories', 'Solid-State Drives'): ('Computer Peripherals & Storage', 'Data Center Servers'), ('Computing & Data Storage Categories', 'Tablet PCs'): ('Computer Peripherals & Storage', 'Tablet PCs'), ('Computing & Data Storage Categories', 'USB Flash Drive'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'), ('Consumer Electronics Categories', 'Fitness & Wellness Wearable Electronics'): ('Computer Platforms', 'Test & Measurement'), ('Consumer Electronics Categories', 'LCD TV'): ('Computer Platforms', 'TV'), ('Consumer Electronics Categories', 'Major Home Appliances'): ('Computer Platforms', 'Appliance'), ('Consumer Electronics Categories', 'OLED TV'): ('Computer Platforms', 'TV'), ('Consumer Electronics Categories', 'Other Audio/Video'): ('Computer Platforms', 'Other Storage'), ('Consumer Electronics Categories', 'Other Consumer Electronics'): ('Computer Platforms', 'Other Consumer'), ('Consumer Electronics Categories', 'Set-Top Boxes'): ('Computer Platforms', 'Desktop PCs'), ('Consumer Electronics Categories', 'Smart Speakers & Digital Assistants'): ('Computer Platforms', 'Smart Cards'), ('Consumer Electronics Categories', 'Smart Watches'): ('Computer Platforms', 'Smart Cards'), ('Consumer Electronics Categories', 'VR Headsets'): ('Computer Platforms', 'Handset'), ('Consumer Electronics Categories', 'Video Game Consoles'): ('Computer Platforms', 'Video Games'), ('Industrial Electronics Categories', 'Automation'): ('Industrial', 'Auto Powertrain'), ('Industrial Electronics Categories', 'Building & Home Control'): ('Industrial', 'Manufacturing Equipment'), ('Industrial Electronics Categories', 'Lighting'): ('Industrial', 'Manufacturing Equipment'), ('Industrial Electronics Categories', 'Medical Electronics'): ('Industrial', 'Media Tablets'), ('Industrial Electronics Categories', 'Military & Civil Aerospace'): ('Industrial', 'Military/Aerospace'), ('Industrial Electronics Categories', 'Other Industrial'): ('Industrial', 'Other Industrial'), ('Industrial Electronics Categories', 'Power & Energy'): ('Industrial', 'Power & Energy'), ('Industrial Electronics Categories', 'Security & Video Surveillance'): ('Industrial', 'Test & Measurement'), ('Industrial Electronics Categories', 'Test & Measurement'): ('Industrial', 'Test & Measurement'), ('Wired Communications Categories', 'Broadcast & Streaming Video'): ('Wired Communications', 'Test & Measurement'), ('Wired Communications Categories', 'Cable Aggregation Equipment'): ('Wired Communications', 'Manufacturing Equipment'), ('Wired Communications Categories', 'Cable CPE'): ('Wired Communications', 'Tablet PCs'), ('Wired Communications Categories', 'Carrier Ethernet Switches & Routers'): ('Wired Communications', 'Other Computer Products'), ('Wired Communications Categories', 'DSL Aggregation Equipment'): ('Wired Communications', 'Manufacturing Equipment'), ('Wired Communications Categories', 'DSL CPE'): ('Wired Communications', 'Desktop PCs'), ('Wired Communications Categories', 'Data Center Network Switches'): ('Wired Communications', 'Data Center Servers'), ('Wired Communications Categories', 'Enterprise Ethernet Switches & Routers'): ('Wired Communications', 'Enterprise/SOHO'), ('Wired Communications Categories', 'Enterprise UC & Voice'): ('Wired Communications', 'Enterprise/SOHO'), ('Wired Communications Categories', 'FTTH Aggregation Equipment'): ('Wired Communications', 'Manufacturing Equipment'), ('Wired Communications Categories', 'FTTH CPE'): ('Wired Communications', 'Tablet PCs'), ('Wired Communications Categories', 'Low-Tier Consumer/SOHO Routers'): ('Wired Communications', 'Other Consumer'), ('Wired Communications Categories', 'Optical Equipment'): ('Wired Communications', 'Manufacturing Equipment'), ('Wired Communications Categories', 'Other Wired Communications'): ('Wired Communications', 'Other Wired'), ('Wired Communications Categories', 'Threat Mitigation Products'): ('Wired Communications', 'DC Network & Threat Mitigation'), ('Wireless Communications Categories', 'Gray Market Handsets'): ('Wireless Communications', 'Handset'), ('Wireless Communications Categories', 'High-Tier Smartphone'): ('Wireless Communications', 'Other Peripherals'), ('Wireless Communications Categories', 'Low-Tier Smartphone'): ('Wireless Communications', 'Power & Energy'), ('Wireless Communications Categories', 'M2M Modules'): ('Wireless Communications', 'Media Tablets'), ('Wireless Communications Categories', 'Media Tablets'): ('Wireless Communications', 'Media Tablets'), ('Wireless Communications Categories', 'Mid-Tier Smartphone'): ('Wireless Communications', 'Video Games'), ('Wireless Communications Categories', 'Mobile Comm Infrastructure'): ('Wireless Communications', 'Infrastructure'), ('Wireless Communications Categories', 'Mobile Phone (ULCH, Entry, Feature)'): ('Wireless Communications', 'Military/Aerospace'), ('Wireless Communications Categories', 'Other Wireless Communications'): ('Wireless Communications', 'Other Wireless'), ('Wireless Communications Categories', 'Wireless LAN Equipment'): ('Wireless Communications', 'Manufacturing Equipment')}

In [212]:
#Nb of keys
print(len(dict_map.keys()))

#Rename the market and submarket

def remapping(market, submarket):
    key = (market, submarket)
    return mapping_dict[key][0], mapping_dict[key][1]

market_spend.Market, market_spend.Submarket = zip(*market_spend.apply(lambda x : remapping(x.Market, x.Submarket), axis = 1))
market_spend.rename(columns = {"Submarket" : "SubMarket"}, inplace = True)
market_spend

65


,Market,SubMarket,Device,Year,Region,Market size
0,Automotive,Auto ADAS,Amplifier/Comparator,2008,Americas,0.424
1,Automotive,Auto ADAS,Amplifier/Comparator,2008,Asia & Oceania (excl. Japan),0.246
2,Automotive,Auto ADAS,Amplifier/Comparator,2008,EMEA,0.678
3,Automotive,Auto ADAS,Amplifier/Comparator,2008,Japan,0.421
4,Automotive,Auto ADAS,Amplifier/Comparator,2008,Worldwide,1.769
...,...,...,...,...,...,...
32495,Wireless Communications,Infrastructure,Voltage Regulator/Reference,2027,Americas,0.000
32496,Wireless Communications,Infrastructure,Voltage Regulator/Reference,2027,Asia & Oceania (excl. Japan),0.000
32497,Wireless Communications,Infrastructure,Voltage Regulator/Reference,2027,EMEA,0.000
32498,Wireless Communications,Infrastructure,Voltage Regulator/Reference,2027,Japan,0.000


In [197]:
market_spend = market_spend[(market_spend.Region == "Worldwide")]
market_spend.drop("Region", axis = 1, inplace = True)
market_spend

<ipython-input-197-612f2c74d230>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_spend.drop("Region", axis = 1, inplace = True)


,Market,SubMarket,Device,Year,Market size
4,Automotive,Auto ADAS,Amplifier/Comparator,2008,1.769
9,Automotive,Auto ADAS,Amplifier/Comparator,2009,1.390
14,Automotive,Auto ADAS,Amplifier/Comparator,2010,3.401
19,Automotive,Auto ADAS,Amplifier/Comparator,2011,5.911
24,Automotive,Auto ADAS,Amplifier/Comparator,2012,7.250
...,...,...,...,...,...
32479,Wireless Communications,Infrastructure,Voltage Regulator/Reference,2023,0.000
32484,Wireless Communications,Infrastructure,Voltage Regulator/Reference,2024,0.000
32489,Wireless Communications,Infrastructure,Voltage Regulator/Reference,2025,0.000
32494,Wireless Communications,Infrastructure,Voltage Regulator/Reference,2026,0.000


In [213]:
marketspendings = market_spend[["Market", "Year", "Market size"]].groupby(["Market", "Year"]).sum().reset_index()
submarketspendings = market_spend[["SubMarket", "Year", "Market size"]].groupby(["SubMarket", "Year"]).sum().reset_index()
devicespendings = market_spend[["Device", "Year", "Market size"]].groupby(["Device", "Year"]).sum().reset_index()

In [214]:
new_pred = new_pred.merge(market_spend, how = 'left', on = ['Market', "SubMarket", "Device", "Year"])
new_pred = new_pred.groupby(["Company", "Device", "Market", "SubMarket", "Year"]).sum().reset_index()
new_pred.rename(columns = {"Market size" : "Spendings"}, inplace = True)
new_pred = new_pred[new_pred['Spendings'] != 0]
new_pred

,Company,Device,Market,SubMarket,Year,Cogs,Cogs_1,Cogs_2,Revenue,Revenue_1,Revenue_2,Ebitda,Ebitda_1,Ebitda_2,Spendings
0,ABB,Amplifier/Comparator,Industrial,Power & Energy,2024,50610.989286,49398.392857,98560.000000,164851.75,160872.60,147230.00,29718.25,29038.400000,22385.000000,645.606
1,ABB,Amplifier/Comparator,Industrial,Power & Energy,2025,51823.585714,50610.989286,49398.392857,172553.90,164851.75,160872.60,31356.20,29718.250000,29038.400000,698.312
2,ABB,Amplifier/Comparator,Industrial,Power & Energy,2026,53036.182143,51823.585714,50610.989286,184175.00,172553.90,164851.75,34985.00,31356.200000,29718.250000,774.840
3,ABB,Amplifier/Comparator,Industrial,Power & Energy,2027,54248.778571,53036.182143,51823.585714,193501.25,184175.00,172553.90,36050.00,34985.000000,31356.200000,870.598
4,ABB,Analog Application Specific IC,Industrial,Power & Energy,2024,50610.989286,49398.392857,98560.000000,164851.75,160872.60,147230.00,29718.25,29038.400000,22385.000000,1796.096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11494,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Infrastructure,2027,103892.642857,96382.889286,88873.135714,63640.00,59800.00,52608.80,14930.00,13600.000000,11710.300000,498.002
11495,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2024,81363.382143,73853.628571,31570.000000,46718.80,45367.60,57810.00,9167.60,36688.857143,11400.000000,909.998
11496,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2025,88873.135714,81363.382143,73853.628571,52608.80,46718.80,45367.60,11710.30,9167.600000,36688.857143,996.000
11497,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2026,96382.889286,88873.135714,81363.382143,59800.00,52608.80,46718.80,13600.00,11710.300000,9167.600000,1010.002


In [215]:
new_pred = new_pred.merge(marketspendings, how = 'left', on = ["Market", "Year"])
new_pred.rename(columns = {"Market size" : "MarketSpends"}, inplace = True)
new_pred = new_pred.merge(submarketspendings, how = 'left', on = ["SubMarket", "Year"])
new_pred.rename(columns = {"Market size" : "SubMarketSpends"}, inplace = True)
new_pred = new_pred.merge(devicespendings, how = 'left', on = ["Device", "Year"])
new_pred.rename(columns = {"Market size" : "DevSpends"}, inplace = True)
new_pred

,Company,Device,Market,SubMarket,Year,Cogs,Cogs_1,Cogs_2,Revenue,Revenue_1,Revenue_2,Ebitda,Ebitda_1,Ebitda_2,Spendings,MarketSpends,SubMarketSpends,DevSpends
0,ABB,Amplifier/Comparator,Industrial,Power & Energy,2024,50610.989286,49398.392857,98560.000000,164851.75,160872.60,147230.00,29718.25,29038.400000,22385.000000,645.606,35157.968,6962.408,10450.000
1,ABB,Amplifier/Comparator,Industrial,Power & Energy,2025,51823.585714,50610.989286,49398.392857,172553.90,164851.75,160872.60,31356.20,29718.250000,29038.400000,698.312,37436.872,7842.946,11282.018
2,ABB,Amplifier/Comparator,Industrial,Power & Energy,2026,53036.182143,51823.585714,50610.989286,184175.00,172553.90,164851.75,34985.00,31356.200000,29718.250000,774.840,40124.976,8895.592,11658.014
3,ABB,Amplifier/Comparator,Industrial,Power & Energy,2027,54248.778571,53036.182143,51823.585714,193501.25,184175.00,172553.90,36050.00,34985.000000,31356.200000,870.598,43180.226,9867.924,12070.002
4,ABB,Analog Application Specific IC,Industrial,Power & Energy,2024,50610.989286,49398.392857,98560.000000,164851.75,160872.60,147230.00,29718.25,29038.400000,22385.000000,1796.096,35157.968,6962.408,103910.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Infrastructure,2027,103892.642857,96382.889286,88873.135714,63640.00,59800.00,52608.80,14930.00,13600.000000,11710.300000,498.002,56945.088,6252.006,40108.022
11445,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2024,81363.382143,73853.628571,31570.000000,46718.80,45367.60,57810.00,9167.60,36688.857143,11400.000000,909.998,53294.638,8689.998,33658.010
11446,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2025,88873.135714,81363.382143,73853.628571,52608.80,46718.80,45367.60,11710.30,9167.600000,36688.857143,996.000,58894.658,9542.002,36652.024
11447,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2026,96382.889286,88873.135714,81363.382143,59800.00,52608.80,46718.80,13600.00,11710.300000,9167.600000,1010.002,57571.344,9351.998,38270.010


In [216]:
def log_diff(x, y):
    return np.log(1+x) - np.log(1+y)

In [220]:
new_pred["Spendings_1"] = new_pred.groupby(["Company", "Device", "Market", "SubMarket"]).Spendings.shift(1)
new_pred["MarketSpends_1"] = new_pred.groupby(["Company", "Device", "Market", "SubMarket"]).MarketSpends.shift(1)
new_pred["SubMarketSpends_1"] = new_pred.groupby(["Company", "Device", "Market", "SubMarket"]).SubMarketSpends.shift(1)
new_pred["DevSpends_1"] = new_pred.groupby(["Company", "Device", "Market", "SubMarket"]).DevSpends.shift(1)

new_pred["LogDiffSpendings"] = new_pred.apply(lambda x: log_diff(x.Spendings, x.Spendings_1), axis = 1)
new_pred["LogDiffMarketSpends"] = new_pred.apply(lambda x: log_diff(x.MarketSpends, x.MarketSpends_1), axis = 1)
new_pred["LogDiffSubMarketSpends"] = new_pred.apply(lambda x: log_diff(x.SubMarketSpends, x.SubMarketSpends_1), axis = 1)
new_pred["LogDiffDevSpends"] = new_pred.apply(lambda x: log_diff(x.DevSpends, x.DevSpends_1), axis = 1)

new_pred.drop(["Spendings", "Spendings_1", "MarketSpends", "MarketSpends_1", "SubMarketSpends", "SubMarketSpends_1", "DevSpends", "DevSpends_1"], axis = 1, inplace = True)

new_pred["LogDiffCogs"] = new_pred.apply(lambda x: log_diff(x.Cogs, x.Cogs_1), axis = 1)
new_pred["LogDiffRevenue"] = new_pred.apply(lambda x: log_diff(x.Revenue, x.Revenue_1), axis = 1)
new_pred["LogDiffEbitda"] = new_pred.apply(lambda x: log_diff(x.Ebitda, x.Ebitda_1), axis = 1)
new_pred["LogDiffCogs_1"] = new_pred.apply(lambda x: log_diff(x.Cogs_1, x.Cogs_2), axis = 1)
new_pred["LogDiffRevenue_1"] = new_pred.apply(lambda x: log_diff(x.Revenue_1, x.Revenue_2), axis = 1)
new_pred["LogDiffEbitda_1"] = new_pred.apply(lambda x: log_diff(x.Ebitda_1, x.Ebitda_2), axis = 1)

new_pred.drop(["Cogs_1", "Cogs_2", "Cogs", "Revenue", "Revenue_1", "Revenue_2", "Ebitda", "Ebitda_1", "Ebitda_2"], axis = 1, inplace = True)
new_pred

,Company,Device,Market,SubMarket,Year,LogDiffSpendings,LogDiffMarketSpends,LogDiffSubMarketSpends,LogDiffDevSpends,LogDiffCogs,LogDiffRevenue,LogDiffEbitda,LogDiffCogs_1,LogDiffRevenue_1,LogDiffEbitda_1
0,ABB,Amplifier/Comparator,Industrial,Power & Energy,2024,NaN,NaN,NaN,NaN,0.024250,0.024434,0.023141,-0.690738,0.088616,0.260218
1,ABB,Amplifier/Comparator,Industrial,Power & Energy,2025,0.078360,0.062803,0.119073,0.076601,0.023676,0.045663,0.053649,0.024250,0.024434,0.023141
2,ABB,Amplifier/Comparator,Industrial,Power & Energy,2026,0.103849,0.069341,0.125926,0.032781,0.023129,0.065176,0.109504,0.023676,0.045663,0.053649
3,ABB,Amplifier/Comparator,Industrial,Power & Energy,2027,0.116382,0.073382,0.103723,0.034726,0.022606,0.049397,0.029987,0.023129,0.065176,0.109504
4,ABB,Analog Application Specific IC,Industrial,Power & Energy,2024,NaN,NaN,NaN,NaN,0.024250,0.024434,0.023141,-0.690738,0.088616,0.260218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Infrastructure,2027,0.016158,-0.010937,-0.007648,0.046909,0.075029,0.062236,0.093296,0.081118,0.128120,0.149589
11445,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2024,NaN,NaN,NaN,NaN,0.096839,0.029348,-1.386716,0.849860,-0.242359,1.168799
11446,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2025,0.090210,0.099913,0.093520,0.085215,0.088284,0.118734,0.244770,0.096839,0.029348,-1.386716
11447,Zebra Technologies,Voltage Regulator/Reference,Wireless Communications,Other Wireless,2026,0.013946,-0.022725,-0.020111,0.043197,0.081118,0.128120,0.149589,0.088284,0.118734,0.244770


In [222]:
new_pred = new_pred[~new_pred['LogDiffSpendings'].isnull()]
new_pred.to_csv("../clean_data/new_pred.csv")